# What we need:
1) lib folder for container libraries<br>
2) file handling mechanism<br>
3) Exception handling<br>

In [ ]:
import time
from watchdog.observers import Observer
from watchdog.events import PatternMatchingEventHandler

# Event handler
patterns = "*"
ignore_patterns = ""
ignore_directories = False
case_sensitive = False
automation_event_handler = PatternMatchingEventHandler(patterns,ignore_patterns,ignore_directories,case_sensitive)

def on_created(event):
    print(f"Event, {event.src_path} has been created")

def on_deleted(event):
    print(f"Event, {event.src_path} has been deleted")

automation_event_handler.on_created = on_created
automation_event_handler.on_deleted = on_deleted

path="outputs"
go_recursively = True
automation_observer = Observer()
automation_observer.schedule(automation_event_handler, path, recursive=go_recursively)
automation_observer.start()
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    automation_observer.stop()
    automation_observer.join()